In [1]:
import kagglehub
import os
import zipfile
import pandas as pd
from mlxtend.frequent_patterns import apriori,association_rules

c:\Users\priya\anaconda3\envs\chatbot\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Change the working directory
new_directory = "a:/NLP Projects/Chatbot/python_code/Dataset"  # Replace with your desired path
os.chdir(new_directory)

In [11]:
product = pd.read_csv('products.csv')
sales = pd.read_csv('sales.csv')

In [12]:
product.tail(2)

,product_id,product_group,product_category,product_type,product,product_description,unit_of_measure,current_wholesale_price,current_retail_price,tax_exempt_yn,promo_yn,new_product_yn
38,39,Fertilizers,Organic Fertilizers,Bio-fertilizer,Dec-Mori (Bio-fertilizer),A bio-fertilizer for enhanced plant growth.,1 kg,3.5,$4.38,Y,N,N
39,40,Fertilizers,Organic Fertilizers,Organic Pest Control,Agni Shield,An organic pest control solution.,1 liter,5,$6.25,Y,N,N


In [14]:
sales.head(2)

,transaction_id,transaction_date,transaction_time,sales_outlet_id,staff_id,customer_id,instore_yn,order,line_item_id,product_id,quantity,line_item_amount,unit_price,promo_item_yn
0,11,4/1/2019,15:54:39,3,17,781,N,1,1,27,2,7.0,3.5,N
1,32,4/1/2019,16:06:04,3,12,683,N,1,1,23,2,5.0,2.5,N


In [15]:
product = product[['product_id','product_group','product_category','product']]
sales = sales[['transaction_id','transaction_date','customer_id','product_id','quantity']]

In [16]:
data = pd.merge(sales,product,on='product_id', how='left')

In [18]:
data.head(2)

,transaction_id,transaction_date,customer_id,product_id,quantity,product_group,product_category,product
0,11,4/1/2019,781,27,2,Plants,Cacti & Succulents,Pincushion Cactus
1,32,4/1/2019,683,23,2,Plants,Flowering Plants,Marigold (Orange)


In [21]:
data.isna().sum()

transaction_id      0
transaction_date    0
customer_id         0
product_id          0
quantity            0
product_group       0
product_category    0
product             0
dtype: int64

In [20]:
tran_count = data['transaction_id'].value_counts().reset_index()
tran_count

,transaction_id,count
0,85,32
1,117,31
2,264,30
3,146,29
4,385,29
...,...,...
3395,4090,1
3396,4124,1
3397,4138,1
3398,4150,1


In [25]:
valid_tran = tran_count[(tran_count['count']>1)]['transaction_id'].tolist()
valid_tran[:10]

[85, 117, 264, 146, 385, 55, 148, 236, 118, 303]

In [43]:
final_data = data[data['transaction_id'].isin(valid_tran)]
final_data

,transaction_id,transaction_date,customer_id,product_id,quantity,product_group,product_category,product
0,11,4/1/2019,781,27,2,Plants,Cacti & Succulents,Pincushion Cactus
1,32,4/1/2019,683,23,2,Plants,Flowering Plants,Marigold (Orange)
2,33,4/1/2019,99,34,1,Plants,Cacti & Succulents,Aloevera
3,39,4/1/2019,664,32,1,Plants,Cacti & Succulents,Graptoveria opalina - Succulent Plant
4,81,4/1/2019,35,35,2,Plants,Cacti & Succulents,Euphorbia (Red)
...,...,...,...,...,...,...,...,...
19111,736,4/29/2019,0,38,1,Fertilizers,Organic Fertilizers,Dec-Neemo (Bio-fertilizer)
19112,746,4/29/2019,0,37,2,Fertilizers,Organic Fertilizers,Vermicompost Mixture
19113,753,4/29/2019,0,30,1,Plants,Cacti & Succulents,Golden Pipe Cactus
19114,756,4/29/2019,8412,25,2,Plants,Flowering Plants,Rajnigandha


## Popularity Recomendation

In [27]:
pop_rec_data = final_data.groupby(['product_category','product']).count().reset_index()
pop_rec_data = pop_rec_data[['product_category','product','transaction_id']]
pop_rec_data = pop_rec_data.rename(columns={'transaction_id':'count'})
pop_rec_data = pop_rec_data[pop_rec_data['count']>1]
pop_rec_data

,product_category,product,count
0,Cacti & Succulents,Aloevera,828
1,Cacti & Succulents,Bunny Ear Cactus,865
2,Cacti & Succulents,Crassula tetragona - Succulent Plant,893
3,Cacti & Succulents,Echinopsis chamaecereus – Cactus,950
4,Cacti & Succulents,Euphorbia (Red),836
5,Cacti & Succulents,Golden Pipe Cactus,880
6,Cacti & Succulents,Graptoveria opalina - Succulent Plant,937
7,Cacti & Succulents,Moon Cactus (Grafted,893
8,Cacti & Succulents,Pincushion Cactus,867
9,Flowering Plants,Champa (White),859


In [28]:
# Change the working directory
new_directory = "a:/NLP Projects/Chatbot/python_code"  # Replace with your desired path
os.chdir(new_directory)

pop_rec_data.to_csv('api/recomendation_objects/popularity_recomendation.csv',index=False)

## Apriori Engine

In [44]:
final_data.head(2)

,transaction_id,transaction_date,customer_id,product_id,quantity,product_group,product_category,product
0,11,4/1/2019,781,27,2,Plants,Cacti & Succulents,Pincushion Cactus
1,32,4/1/2019,683,23,2,Plants,Flowering Plants,Marigold (Orange)


In [ ]:
#final_data['transaction'] = final_data['transaction_id'].astype(str) +"_"+  final_data['customer_id'].astype(str)
#final_data.drop(['transaction_id'],axis=1,inplace=True)

C:\Users\priya\AppData\Local\Temp\ipykernel_6936\262457892.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_data.drop(['transaction_id'],axis=1,inplace=True)


In [45]:
final_data.head(2)

,transaction_id,transaction_date,customer_id,product_id,quantity,product_group,product_category,product
0,11,4/1/2019,781,27,2,Plants,Cacti & Succulents,Pincushion Cactus
1,32,4/1/2019,683,23,2,Plants,Flowering Plants,Marigold (Orange)


In [46]:
train_basket = final_data.groupby(['transaction_id','product'])['product'].count().reset_index(name='count')
train_basket.head()

,transaction_id,product,count
0,1,Aloevera,1
1,1,Crassula tetragona - Succulent Plant,1
2,1,Dec-Neemo (Bio-fertilizer),1
3,1,Echinopsis chamaecereus – Cactus,2
4,1,Euphorbia (Red),2


In [47]:
my_basket = train_basket.pivot_table(index='transaction_id',columns='product',values='count',fill_value=0)
my_basket.head()

product,Aglaonema Lipstick,Agni Shield,Ajwain Leaves,Aloevera,Bhuiamla,Black Turmeric Plant (Black Haldi),Bunny Ear Cactus,Champa (White),Chlorophytum Spider Plant,Crassula tetragona - Succulent Plant,...,Rajnigandha,Rama Tulsi Plant,Rose,Rubber Tree (Ficus elastica),Shevanti,Snake Plant (Sansevieria),Vermicompost,Vermicompost Mixture,White Butterfly (Syngonium Podophyllum),Wild Asparagus
transaction_id,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,1,0,0,0,0,0,1,...,1,0,0,0,1,1,0,2,0,0
2,0,1,0,0,0,0,0,2,0,1,...,3,0,0,0,0,0,2,0,0,0
3,0,1,0,1,0,0,0,1,0,0,...,0,0,0,1,3,0,0,2,1,0
4,0,1,0,0,0,0,0,1,0,2,...,0,0,0,0,2,0,1,0,0,0
5,0,2,0,0,0,0,1,2,0,2,...,0,0,0,0,0,0,0,0,0,0


In [48]:
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

In [49]:
my_basket = my_basket.applymap(encode_units)
my_basket.head()

product,Aglaonema Lipstick,Agni Shield,Ajwain Leaves,Aloevera,Bhuiamla,Black Turmeric Plant (Black Haldi),Bunny Ear Cactus,Champa (White),Chlorophytum Spider Plant,Crassula tetragona - Succulent Plant,...,Rajnigandha,Rama Tulsi Plant,Rose,Rubber Tree (Ficus elastica),Shevanti,Snake Plant (Sansevieria),Vermicompost,Vermicompost Mixture,White Butterfly (Syngonium Podophyllum),Wild Asparagus
transaction_id,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,1,0,0,0,0,0,1,...,1,0,0,0,1,1,0,1,0,0
2,0,1,0,0,0,0,0,1,0,1,...,1,0,0,0,0,0,1,0,0,0
3,0,1,0,1,0,0,0,1,0,0,...,0,0,0,1,1,0,0,1,1,0
4,0,1,0,0,0,0,0,1,0,1,...,0,0,0,0,1,0,1,0,0,0
5,0,1,0,0,0,0,1,1,0,1,...,0,0,0,0,0,0,0,0,0,0


In [50]:
frequent_items = apriori(my_basket, min_support=0.05, use_colnames=True)
frequent_items.head(10)

c:\Users\priya\anaconda3\envs\chatbot\Lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:161: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,support,itemsets
0,0.281792,(Agni Shield)
1,0.261607,(Aloevera)
2,0.265240,(Bunny Ear Cactus)
3,0.256762,(Champa (White))
4,0.279370,(Crassula tetragona - Succulent Plant)
5,0.280581,(Dec-Mori (Bio-fertilizer))
6,0.273314,(Dec-Neemo (Bio-fertilizer))
7,0.293096,(Echinopsis chamaecereus – Cactus)
8,0.249899,(Euphorbia (Red))
9,0.275333,(Fragrant Panama rose)


In [51]:
rules_basket = association_rules(frequent_items, metric="lift", min_threshold=1)
rules_basket.head(10)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
0,(Agni Shield),(Aloevera),0.281792,0.261607,0.094469,0.335244,1.281479,1.0,0.020750,1.110773,0.305833,0.210432,0.099726,0.348177
1,(Aloevera),(Agni Shield),0.261607,0.281792,0.094469,0.361111,1.281479,1.0,0.020750,1.124151,0.297472,0.210432,0.110440,0.348177
2,(Agni Shield),(Bunny Ear Cactus),0.281792,0.265240,0.095277,0.338109,1.274727,1.0,0.020534,1.110092,0.300078,0.210903,0.099173,0.348659
3,(Bunny Ear Cactus),(Agni Shield),0.265240,0.281792,0.095277,0.359209,1.274727,1.0,0.020534,1.120813,0.293318,0.210903,0.107791,0.348659
4,(Agni Shield),(Champa (White)),0.281792,0.256762,0.096891,0.343840,1.339136,1.0,0.024538,1.132707,0.352614,0.219378,0.117159,0.360599
5,(Champa (White)),(Agni Shield),0.256762,0.281792,0.096891,0.377358,1.339136,1.0,0.024538,1.153485,0.340739,0.219378,0.133062,0.360599
6,(Agni Shield),(Crassula tetragona - Succulent Plant),0.281792,0.279370,0.106581,0.378223,1.353843,1.0,0.027856,1.158985,0.363909,0.234458,0.137176,0.379863
7,(Crassula tetragona - Succulent Plant),(Agni Shield),0.279370,0.281792,0.106581,0.381503,1.353843,1.0,0.027856,1.161214,0.362686,0.234458,0.138832,0.379863
8,(Agni Shield),(Dec-Mori (Bio-fertilizer)),0.281792,0.280581,0.108195,0.383954,1.368424,1.0,0.029130,1.167801,0.374867,0.238222,0.143689,0.384783
9,(Dec-Mori (Bio-fertilizer)),(Agni Shield),0.280581,0.281792,0.108195,0.385612,1.368424,1.0,0.029130,1.168979,0.374236,0.238222,0.144553,0.384783


In [53]:
rules_basket[rules_basket['antecedents'] == {'Agni Shield'}].sort_values(by='confidence',ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
20,(Agni Shield),(Graptoveria opalina - Succulent Plant),0.281792,0.292289,0.113444,0.402579,1.377331,1.0,0.031079,1.184610,0.381447,0.246275,0.155840,0.395350
10,(Agni Shield),(Dec-Neemo (Bio-fertilizer)),0.281792,0.273314,0.109407,0.388252,1.420533,1.0,0.032389,1.187884,0.412191,0.245471,0.158167,0.394274
30,(Agni Shield),(Shevanti),0.281792,0.281389,0.109003,0.386819,1.374680,1.0,0.029710,1.171941,0.379497,0.240000,0.146715,0.387097
8,(Agni Shield),(Dec-Mori (Bio-fertilizer)),0.281792,0.280581,0.108195,0.383954,1.368424,1.0,0.029130,1.167801,0.374867,0.238222,0.143689,0.384783
34,(Agni Shield),(Vermicompost Mixture),0.281792,0.275333,0.107792,0.382521,1.389305,1.0,0.030205,1.173591,0.390159,0.239892,0.147914,0.387009
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
399,(Agni Shield),"(Crassula tetragona - Succulent Plant, Echinop...",0.281792,0.102543,0.050464,0.179083,1.746413,1.0,0.021568,1.093237,0.595089,0.151149,0.085285,0.335605
681,(Agni Shield),"(Vermicompost Mixture, Marigold (Orange))",0.281792,0.100525,0.050061,0.177650,1.767229,1.0,0.021733,1.093787,0.604481,0.150668,0.085745,0.337821
357,(Agni Shield),"(Crassula tetragona - Succulent Plant, Bunny E...",0.281792,0.094065,0.050061,0.177650,1.888584,1.0,0.023554,1.101642,0.655107,0.153656,0.092264,0.354920
363,(Agni Shield),"(Bunny Ear Cactus, Moon Cactus (Grafted)",0.281792,0.101736,0.050061,0.177650,1.746191,1.0,0.021392,1.092314,0.594988,0.150121,0.084512,0.334857


In [57]:
product_category = final_data[['product','product_category']].drop_duplicates().set_index('product').to_dict()['product_category']

In [58]:
recommendations_json = {}

antecedents = rules_basket['antecedents'].unique()
for antecedent in antecedents:
    df_rec = rules_basket[rules_basket['antecedents']==antecedent]
    df_rec = df_rec.sort_values('confidence',ascending=False)
    key = "_".join(antecedent)
    recommendations_json[key] = []
    for _, row in df_rec.iterrows():
        rec_objects =row['consequents']
        for rec_object in rec_objects:
            already_exists = False
            for current_rec_object in recommendations_json[key]:
                if rec_object == current_rec_object['product']:
                    already_exists=True
            if already_exists:
                continue
            
            rec = {'product':rec_object, 
                   'product_category':product_category[rec_object],
                   'confidence': row['confidence']
                  }
            recommendations_json[key].append(rec)

In [ ]:
import pprint
pprint.pp(recommendations_json)

In [60]:
import json
with open('api/recomendation_objects/apriori_recomendation.json', 'w') as json_file:
    json.dump(recommendations_json,json_file)